FunctionSet.py contains a set of functions that do things like take finite difference derivatives of grid variables.

Also, numpy is used here so that array adding is easier to do.

In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

In [2]:
import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal

Declaring grid variables and indices. Note that the variables to be evolved are the vector potential and electric field.

Attaching grid indices to the declared variables.  

Also... 'Low' is meant to show that this variable is a vector with a lower index.   This is unimportant in flat space here, but will be necessary to keep track of when a curved metric is used in future developments.

In [3]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

var_A = ['Ax','Ay','Az']
var_E = ['Ex','Ey','Ez']
var_Gamma = ['Gamma']
var_phi = ['phi']
var_names = var_A+var_E+var_Gamma+var_phi

ELow = ixp.zerorank1(DIM=3)
ALow = ixp.zerorank1(DIM=3)
for itr in range(len(directions)):
    ELow[itr] = IndexedBase('state_fab'+directions[itr])[i,j,k,var_E[itr]]
    ALow[itr] = IndexedBase('state_fab'+directions[itr])[i,j,k,var_A[itr]]
ELow = np.array(ELow)
ALow = np.array(ALow)

Gamma =IndexedBase('state_fab')[i,j,k,var_Gamma[0]]

phi = IndexedBase('state_fab')[i,j,k,var_phi[0]]

Righthand side of Maxwell evolution equations written in a symbolic form.  These are in vacuum for now, but can easily be extended to include currents.

In [4]:
RHS_Gamma = -Lap(phi)
RHS_dotALow = -ELow-np.array(grad(phi))
RHS_dotELow = -LapTen(ALow)+grad(Gamma)

Making use of the AMReXcode to convert RHSs to compilable code blocks to be inserted in the AMReX advance.cpp code. You can choose to prepend the expression with the updated "new" fabs.

In [5]:
AMRoutGamma = AMReXcode(RHS_Gamma,var_names,True,var_Gamma[0])+"\n\n"
print(AMRoutGamma)

AMRoutA = ""
for itr in range(len(directions)):
    AMRoutA += AMReXcode(RHS_dotALow[itr],var_names,True,var_A[itr]) + "\n\n"
print(AMRoutA)

AMRoutE = ""
for itr in range(len(directions)):
    AMRoutE += AMReXcode(RHS_dotELow[itr],var_names,True,var_E[itr]) + "\n\n"
print(AMRoutE)

rhs_fab(i, j, k, Idx::Gamma) = -(state_fab(i, j, k - 1, Idx::phi) + state_fab(i, j, k + 1, Idx::phi) - 2*state_fab(i, j, k, Idx::phi))/(dx[2]*dx[2]) - (state_fab(i, j - 1, k, Idx::phi) + state_fab(i, j + 1, k, Idx::phi) - 2*state_fab(i, j, k, Idx::phi))/(dx[1]*dx[1]) - (state_fab(i - 1, j, k, Idx::phi) + state_fab(i + 1, j, k, Idx::phi) - 2*state_fab(i, j, k, Idx::phi))/(dx[0]*dx[0]);


rhs_fab(i, j, k, Idx::Ax) = -state_fabx(i, j, k, Idx::Ex) - (-state_fab(i - 1, j, k, Idx::phi) + state_fab(i + 1, j, k, Idx::phi))/(2*dx[0]);

rhs_fab(i, j, k, Idx::Ay) = -state_faby(i, j, k, Idx::Ey) - (-state_fab(i, j - 1, k, Idx::phi) + state_fab(i, j + 1, k, Idx::phi))/(2*dx[1]);

rhs_fab(i, j, k, Idx::Az) = -state_fabz(i, j, k, Idx::Ez) - (-state_fab(i, j, k - 1, Idx::phi) + state_fab(i, j, k + 1, Idx::phi))/(2*dx[2]);


rhs_fab(i, j, k, Idx::Ex) = -(state_fabx(i, j, k - 1, Idx::Ax) + state_fabx(i, j, k + 1, Idx::Ax) - 2*state_fabx(i, j, k, Idx::Ax))/(dx[2]*dx[2]) - (state_fabx(i, j - 1, k, Idx::Ax